# Trending FourSquare Venues in COVID-19 Hot Spot Zip Codes in Virginia, USA

## Introduction
According to the Virginia Department of Health COVID-19 is spread through respiratory droplets, and is more likely spread when people are in close contact. "Community spread" is the public health term used to describe someone being infected by a disease when they do not know when they were in contact with the sick person. Community spread may be occuring Virginia communities at indoor public venues such as restaurants, bars, coffee shops, and grocery stores. 

Determining what categories of venues are correlated with high per capita COVID-19 infection rates can be helpful to determine community risk factors. This project uses FourSquare venues by Virginia zip codes to determine if such a correlation exists. Machine Learning, specific the Support Vector Machine method, is used to determine if high COVID-19 per capita infections can be predicted based on types of trending venues by zip code.


## The Data

Four data sources are utilized in this project:

1. **World Population Review** provides a csv of Virginia Zip Codes and estimates of the 2020 population for each zip code.
  * These data are required to acquire a dataset of Virginia zip codes, and to determine the per capita COVID-19 infections per zip code.
  * https://worldpopulationreview.com/zips/virginia
2. **Open Data Standard API** will be used to aquire a latitude and longitude for each zip code.
  * **It should be noted** that the FourSquare API can use zip codes with the ***&near*** querystring parameter, but in order for more practice I will retrieve latitudes and longitudes.
  * https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA
3. **Virginia's Open Data Portal of COVID-19 Counts by Zip Code** provides cummulative COVID-19 counts per Virginia zip code.
  * These data are required to calculate the per capita COVID-19 infections per zip code.
  * Data are included for 5/15/2020 - 12-27-2020: I will use **December 1, 2020 - December 27, 2020** in order to focus on the most current infections, and also to best match venue data which is from December 28, 2020.
  * I subtract total cases for each zip code on November 30th from the total cases on December 27th.
  * I divide the total December cases by zip code population to determine the per capita COVID case count.
  * I then determine the mean per capita case count and assign a categorical label: "low" or "high" to the zip code based on whether the zip code's per capita COVID-19 December cases are above or below the mean.
  * https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-ZIPCode/8bkr-zfqv

4. **FourSquare Venues Endpoint** will provide venues within 2000m (2 km) of the zip code.
  * The FourSquare API will return categories for each venue. **The top 10 categories for each zip code will be calculated, and the resulting data will be used in the Support Vector Machine to determine what accuracy we can predict high or low per capita COVID-19 infections by venue categories.**
  * **It should be noted** that many venues will be beyond the 2000m radius of the FourSquare search. This project is for example purposes only.
  * https://developer.foursquare.com/docs/places-api/endpoints/
  

### Collect the Data

Import needed libraries

In [4]:
# import needed libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#!pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


#### Collect Population For Each Virginia Zip Code

Load the csv of population by Virginia zip code from World Population Review.

In [5]:
df_ZipPop = pd.read_csv(r'Data/VirginiaPopByZipCode.csv')
print(df_ZipPop.shape)
df_ZipPop.head()

(892, 4)


,zip,city,county,pop
0,22193,Woodbridge,Prince William,82573
1,23464,Virginia Beach,Virginia Beach City,76495
2,22191,Woodbridge,Prince William,70641
3,23322,Chesapeake,Chesapeake City,65603
4,20147,Ashburn,Loudoun,64197


Need to change data type of the zip column for later merging.

In [6]:
# need to change data type for later merging
df_ZipPop['zip'] = df_ZipPop['zip'].astype(str)
print(df_ZipPop.dtypes)

zip       object
city      object
county    object
pop        int64
dtype: object


#### Collect Latitude and Longitude for each zip code

From Open Data Standard retrieve all Virginia zip codes and corresponding latitudes and longitudes. Save the results in DataFrame and merge the DataFrame with the population DataFrame.

In [7]:
#https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA

zip_url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA'
            
# make the GET request
zip_results = requests.get(zip_url).json()['records']
#print(zip_results)
zip_latlon_list=[]
zip_latlon_list.append([(z['fields']['zip'], z['fields']['latitude'], z['fields']['longitude']) for z in zip_results])

df_ZipLatLon = pd.DataFrame([item for zip_latlon_list in zip_latlon_list for item in zip_latlon_list])
df_ZipLatLon.columns = ['zip', 'Latitude', 'Longitude']

In [8]:
print(df_ZipLatLon.shape)
df_ZipLatLon['zip'] = df_ZipLatLon['zip'].astype(str)
print(df_ZipLatLon.dtypes)
df_ZipLatLon.head()

(1275, 3)
zip           object
Latitude     float64
Longitude    float64
dtype: object


,zip,Latitude,Longitude
0,23887,36.563755,-77.82652
1,24656,37.198005,-82.12193
2,24149,37.011934,-80.41897
3,23125,37.342721,-76.27989
4,22652,38.823987,-78.45310


* Merge the DataFrames
* Drop NA values from the merged DataFrame

In [9]:
df_ZipUnion = df_ZipLatLon.merge(df_ZipPop, on='zip', how='left')

In [10]:
df_ZipUnion.dropna(inplace=True)
print(df_ZipUnion.shape)
df_ZipUnion.head()

(891, 6)


,zip,Latitude,Longitude,city,county,pop
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0


Create a new **areaName** column in the DataFrame with CITY, COUNTY (ZIPCODE) for display purposes.

In [11]:
df_ZipUnion['areaName'] = df_ZipUnion['city'] + ', ' + df_ZipUnion['county'] + " (" + df_ZipUnion['zip'] + ')'
df_ZipUnion.head()

,zip,Latitude,Longitude,city,county,pop,areaName
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)"
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)"
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)"
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)"
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)"


#### Collect Virginia COVID-19 Infection Counts By Zip Code

Load the csv of Virginia's Open Data Portal of COVID-19 Counts by Zip Code 

In [53]:
df_ZipCOVID = pd.read_csv(r'Data/VDH-COVID-19-PublicUseDataset-ZIPCode.csv')
print(df_ZipCOVID.shape)
df_ZipCOVID.head()

(199668, 5)


,Report Date,ZIP Code,Number of Cases,Number of Testing Encounters,Number of PCR Testing Encounters
0,10/29/2020,20105,327,NaN,6316
1,10/29/2020,20106,78,NaN,792
2,10/29/2020,20109,1868,NaN,12598
3,10/29/2020,20110,2327,NaN,15054
4,10/29/2020,20111,1566,NaN,10305


In [54]:
df_ZipCOVID.dtypes

Report Date                          object
ZIP Code                             object
Number of Cases                      object
Number of Testing Encounters        float64
Number of PCR Testing Encounters     object
dtype: object

Set Report Date to datetime format.

In [55]:
df_ZipCOVID['Report Date'] =  pd.to_datetime(df_ZipCOVID['Report Date'], infer_datetime_format=True)
df_ZipCOVID.dtypes

Report Date                         datetime64[ns]
ZIP Code                                    object
Number of Cases                             object
Number of Testing Encounters               float64
Number of PCR Testing Encounters            object
dtype: object

* I only need the Report Date, Zip Code, and Number of Cases columns.
* Drop NaN records
* rename columns to ReportDate, zip, and NumCases
* I only need the values for dates greater than November 30th.

In [56]:
df_ZipCOVID.drop(['Number of Testing Encounters','Number of PCR Testing Encounters'], axis=1, inplace=True)
df_ZipCOVID.dropna(inplace=True)
df_ZipCOVID = df_ZipCOVID[df_ZipCOVID['Number of Cases'] != 'Suppressed']
df_ZipCOVID = df_ZipCOVID[df_ZipCOVID['Number of Cases'] != 'Suppressed*']
print(df_ZipCOVID.shape)


(169113, 3)


In [57]:
# set Num of Cases to int
df_ZipCOVID['Number of Cases'] =  pd.to_numeric(df_ZipCOVID['Number of Cases'])
df_ZipCOVID.dtypes

Report Date        datetime64[ns]
ZIP Code                   object
Number of Cases             int64
dtype: object

In [58]:
# renaming the columns 
df_ZipCOVID.rename({"Report Date": "ReportDate",  
           "ZIP Code": "zip",  
           "Number of Cases": "NumCases"},  
          axis = "columns", inplace = True) 
print(df_ZipCOVID.shape)
df_ZipCOVID.head()

(169113, 3)


,ReportDate,zip,NumCases
0,2020-10-29,20105,327
1,2020-10-29,20106,78
2,2020-10-29,20109,1868
3,2020-10-29,20110,2327
4,2020-10-29,20111,1566


In [59]:
# remove rows with dates less than 2020-11-30
df_ZipCOVIDDecember = df_ZipCOVID[df_ZipCOVID.ReportDate > '2020-11-30']
print(df_ZipCOVIDDecember.shape)
df_ZipCOVIDDecember.head()

(22760, 3)


,ReportDate,zip,NumCases
31712,2020-12-01,20105,455
31713,2020-12-01,20106,106
31714,2020-12-01,20109,2266
31715,2020-12-01,20110,2703
31716,2020-12-01,20111,1922


In [60]:
# groupby zip code, subtract the minimum NumCases from the maximum NumCases to get the December Number Cases
df_ZIPDecCaseCount = df_ZipCOVIDDecember.groupby('zip').agg({'NumCases': ['min', 'max']}) 
print(df_ZIPDecCaseCount.shape)
df_ZIPDecCaseCount.head()

(867, 2)


NumCases      
           min   max
zip                 
20105      455   643
20106      106   140
20109     2266  2786
20110     2703  3367
20111     1922  2487

In [61]:
#rename columns 
df_ZIPDecCaseCount.columns = ['min', 'max']
# calculate the December case counts
df_ZIPDecCaseCount['DecNumCases'] = df_ZIPDecCaseCount['max'] - df_ZIPDecCaseCount['min']

# drop the min and max columns
df_ZIPDecCaseCount.drop(['min','max'], axis=1, inplace=True)
print(df_ZIPDecCaseCount.shape)
df_ZIPDecCaseCount.head()

(867, 1)


,DecNumCases
zip,
20105,188
20106,34
20109,520
20110,664
20111,565


In [62]:
# reset the index
df_ZIPDecCaseCount.reset_index(inplace=True)
print(df_ZIPDecCaseCount.shape)
df_ZIPDecCaseCount.head()

(867, 2)


,zip,DecNumCases
0,20105,188
1,20106,34
2,20109,520
3,20110,664
4,20111,565


**Merge the Num Cases DataFrame with the df_ZipUnion DataFrame**

In [64]:
print("df_ZipUnion Shape: ",df_ZipUnion.shape)
print("df_ZIPDecCaseCount Shape: ",df_ZIPDecCaseCount.shape)

df_ZipUnion Shape:  (891, 7)
df_ZIPDecCaseCount Shape:  (867, 2)


There are some zip codes with 0 cases. Need to make sure the merge works to not eliminate those zips, and I will need to add a 0 value to the NaNs.

In [65]:
df_ZipCOVIDUnion = df_ZipUnion.merge(df_ZIPDecCaseCount, on='zip', how='left')
df_ZipCOVIDUnion.shape

(891, 8)

In [67]:
df_ZipCOVIDUnion.head(10)

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0
5,22311,38.837312,-77.12064,Alexandria,Alexandria City,18952.0,"Alexandria, Alexandria City (22311)",260.0
6,24174,37.357587,-79.66552,Thaxton,Bedford,2762.0,"Thaxton, Bedford (24174)",18.0
7,23601,37.053346,-76.45948,Newport News,Newport News City,25578.0,"Newport News, Newport News City (23601)",255.0
8,22578,37.644240,-76.36034,White Stone,Lancaster,2315.0,"White Stone, Lancaster (22578)",23.0
9,23389,37.661513,-75.83150,Harborton,Accomack,258.0,"Harborton, Accomack (23389)",NaN


The tenth row has a NaN, need to set this, and all other NaN's to 0.

In [68]:
df_ZipCOVIDUnion.fillna(0, inplace=True)
df_ZipCOVIDUnion.head(10)

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0
5,22311,38.837312,-77.12064,Alexandria,Alexandria City,18952.0,"Alexandria, Alexandria City (22311)",260.0
6,24174,37.357587,-79.66552,Thaxton,Bedford,2762.0,"Thaxton, Bedford (24174)",18.0
7,23601,37.053346,-76.45948,Newport News,Newport News City,25578.0,"Newport News, Newport News City (23601)",255.0
8,22578,37.644240,-76.36034,White Stone,Lancaster,2315.0,"White Stone, Lancaster (22578)",23.0
9,23389,37.661513,-75.83150,Harborton,Accomack,258.0,"Harborton, Accomack (23389)",0.0


### Calculate the per population number of December COVID Cases for each Virginia zip code

In [70]:
df_ZipCOVIDUnion['DecCOVIDPerCap'] = df_ZipCOVIDUnion['DecNumCases'] / df_ZipCOVIDUnion['pop']

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases,DecCOVIDPerCap
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0,0.000000
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0,0.016358
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0,0.014731
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0,0.003922
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0,0.005228
5,22311,38.837312,-77.12064,Alexandria,Alexandria City,18952.0,"Alexandria, Alexandria City (22311)",260.0,0.013719
6,24174,37.357587,-79.66552,Thaxton,Bedford,2762.0,"Thaxton, Bedford (24174)",18.0,0.006517
7,23601,37.053346,-76.45948,Newport News,Newport News City,25578.0,"Newport News, Newport News City (23601)",255.0,0.009970
8,22578,37.644240,-76.36034,White Stone,Lancaster,2315.0,"White Stone, Lancaster (22578)",23.0,0.009935
9,23389,37.661513,-75.83150,Harborton,Accomack,258.0,"Harborton, Accomack (23389)",0.0,0.000000


In [92]:
df_ZipCOVIDUnion.replace([np.inf, -np.inf], np.nan)
df_ZipCOVIDUnion.fillna(0, inplace=True)
df_ZipCOVIDUnion.dtypes

zip                object
Latitude          float64
Longitude         float64
city               object
county             object
pop               float64
areaName           object
DecNumCases       float64
DecCOVIDPerCap    float64
dtype: object

In [95]:
df_ZipCOVIDUnion.describe()


,Latitude,Longitude,pop,DecNumCases,DecCOVIDPerCap
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,37.655585,-78.352604,9307.725028,101.529742,inf
std,0.741489,1.770358,13765.797134,153.625160,NaN
min,36.554164,-83.483220,0.000000,0.000000,0.000000
25%,37.023567,-79.360430,831.000000,7.000000,0.006452
50%,37.528701,-77.998750,2777.000000,31.000000,0.009658
75%,38.234870,-77.100440,11063.500000,134.000000,0.013966
max,39.345906,-75.367800,82573.000000,1232.000000,inf


In [101]:
# calculate the thredhold for above average December Cases Per Capita, vs below average December Cases Per Capita
threshold_DecCasesPerCap = df_ZipCOVIDUnion['DecNumCases'].mean() / df_ZipCOVIDUnion['pop'].mean()
threshold_DecCasesPerCap

0.010908115737950074

### Threshold For Above or Below December COVID-19 Cases Per Capita in Virgina Counties: 1.1% of zip code population

Add a column to the df_ZipCOVIDUnion DataFrame with 'above' or 'below' depending on if the DecCOVIDPerCap is > 0.011 or < 0.011

In [102]:
df_ZipCOVIDUnion['aboveorbelow'] = np.where(df_ZipCOVIDUnion['DecCOVIDPerCap'] < 0.011, 'below', 'above')
df_ZipCOVIDUnion.head()

,zip,Latitude,Longitude,city,county,pop,areaName,DecNumCases,DecCOVIDPerCap,aboveorbelow
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)",0.0,0.000000,below
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)",46.0,0.016358,above
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)",52.0,0.014731,above
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)",1.0,0.003922,below
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)",7.0,0.005228,below


## Methodology 

** TO DO**
Main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

Attempted to use trending venues from FourSquare, but trending venues were unavailable for all of Virginia zip codes for multiple retrival attempts. I had to settle for 50 venues within two kilometers of the zip code location.

I will explore FourSquare trending venues by Virginia zip code. I will merge the trending venues by Virginia zip code with per capita COVID-19 case number increases in the last two weeks in Virginia. I will categorize each zip code as a high rate of COVID per capita, or a low rate of COVID per capita. I will retrieve the top 10 categories of venues for each zip code. I then trained a Support Vector Machine to determine whether a zip code can be predicted to be high or low per capita COVID-19 infections by the categories of trending venues.

## Results

** TO DO**
Discuss the results.

## Discussion

** TO DO**
Discuss any observations you noted and any recommendations you can make based on the results.

## Conclusion

** TO DO**
Conclude the report.

## References
 Virgina Departments of Health: https://www.vdh.virgi
 nia.gov/coronavirus/frequently-asked-questions/general-questions/